# Benchmark for GAT

## Read data

In [1]:
from torch_geometric.datasets import PPI
import torch_geometric.transforms as T
from torch_geometric.utils import add_self_loops
from torch_geometric.loader import DataLoader
import torch 

TRAIN = "train"
VAL = "val"
TEST = "test"
set_names = [TRAIN, TEST, VAL]

train_dataset = PPI(root='/tmp/PPI', split="train")
val_dataset = PPI(root='/tmp/PPI', split="val")
test_dataset = PPI(root='/tmp/PPI', split="test")

train_loader = iter(DataLoader(train_dataset, batch_size=len(train_dataset)))
val_loader = iter(DataLoader(val_dataset, batch_size=len(val_dataset)))
test_loader = iter(DataLoader(test_dataset, batch_size=len(test_dataset)))

device = torch.device("cuda:2") if torch.cuda.is_available() else torch.device("cpu")

train_set = next(train_loader)
test_set = next(test_loader)
val_set = next(val_loader)

sets = dict()
sets[TRAIN] = train_dataset
sets[TEST] = test_dataset
sets[VAL] = val_dataset

/home/dwalke/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
Extracting /tmp/PPI/ppi.zip
Processing...
Done!


In [7]:
train_set

DataBatch(x=[44906, 50], edge_index=[2, 1226368], y=[44906, 121], batch=[44906], ptr=[21])

In [18]:
ds =PPI(root='/tmp/PPI')
next(iter(DataLoader(ds, batch_size=len(ds)))).edge_index.shape

torch.Size([2, 1226368])

## Define GNN architecture

In [4]:
import torch
from torch import nn
from torch_geometric.nn import GATConv
from torch.nn import Linear
import torch.nn.functional as F
class GNN(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, heads, dropout = .2):
        super(GNN, self).__init__()
        self.dropout = dropout
        self.conv1 = GATConv(in_dim, hidden_dim, add_self_loops=True, heads = heads[0], dropout = dropout, concat=True)
        self.lin1 = Linear(in_dim, hidden_dim*heads[0])
        self.conv2 = GATConv(hidden_dim*heads[0], hidden_dim, add_self_loops=True,heads = heads[1], dropout = dropout, concat=True)
        self.lin2 = Linear(hidden_dim*heads[0], hidden_dim*heads[1])
        self.conv3 = GATConv(hidden_dim*heads[1], out_dim, add_self_loops=True,heads = heads[2], dropout = dropout,concat=False)
        self.lin3 = Linear(hidden_dim*heads[1], out_dim)
        
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index) + self.lin1(x)
        x = F.elu(x)
        x = self.conv2(x, edge_index) + self.lin2(x)
        x = F.elu(x)
        x = self.conv3(x, edge_index) + self.lin3(x)
        return x

## Hyperparameter tuning for GNN

In [ ]:
from sklearn.model_selection import ParameterGrid
from tqdm.notebook import tqdm

space = {
    "WEIGHT_DECAYS": [0, 1e-3],
    "HEADS_LIST": [[4,4,6], [6,6,6], [8,8,8], [4,4,4]],
    "DROPOUT": [0.0, 0.2, 0.4],
    "HIDDEN_DIMS": [512, 1024],
    "LEARNING_RATES": [5e-3, 1e-3]
}

param_grid = ParameterGrid(space)
best_params_overall = None
best_val_overall = float("inf")

for params in tqdm(param_grid.__iter__()):    
    gnnTraining = GNNTraining(device = device,
            GNN = GNN,
            sets = sets,
            hidden_dim = params["HIDDEN_DIMS"],
            lr = params["LEARNING_RATES"],
            dropout = params["DROPOUT"],
            weight_decay=params["WEIGHT_DECAYS"],
            epochs = 1000,
            kwargs = {"heads": params["HEADS_LIST"]})
    gnnTraining.train()
    
    if gnnTraining.best_val_loss <= best_val_overall:
        print("Updated params")
        best_val_overall = gnnTraining.best_val_loss
        best_params_overall = params

## Best Hyperparameter

In [40]:
best_params_overall

{'DROPOUT': 0.2,
 'HEADS_LIST': [8, 8, 8],
 'HIDDEN_DIMS': 512,
 'LEARNING_RATES': 0.001,
 'WEIGHT_DECAYS': 0}

## Training & Evaluation

In [6]:
from tqdm.notebook import tqdm
from GNNTraining import GNNTraining
from GNNEvaluate import GNNEvaluate 

gnnTraining = GNNTraining(device = device,
            GNN = GNN,
            sets = sets,
            hidden_dim = 512,
            lr = 1e-3,
            dropout = 0.2,
            weight_decay=0.0,
            epochs = 1000,
            kwargs = {"heads": [8,8,8]})
best_model = gnnTraining.train()

gnnEvaluate = GNNEvaluate(device = device,
            sets = sets)
gnnEvaluate.evaluate(best_model)

  0%|          | 0/1000 [00:00<?, ?it/s]

0.9918850232232057

## Standard deviation over 10 runs

In [8]:
from GNNTraining import GNNTraining
from GNNEvaluate import GNNEvaluate 
from tqdm.notebook import tqdm

times = []
scores = []
for i in tqdm(range(10)):
    gnnTraining = GNNTraining(
            device = device,
            GNN = GNN,
            sets = sets,
            hidden_dim = 512,
            lr = 1e-3,
            dropout = 0.2,
            weight_decay=0.0,
            epochs = 1000,
            kwargs = {"heads": [8,8,8]})
    best_model = gnnTraining.train()
    times.append(gnnTraining.training_time)
    
    gnnEvaluate = GNNEvaluate(device = device,
                sets = sets)
    score = gnnEvaluate.evaluate(best_model)
    scores.append(score)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

...

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [9]:
import numpy as np
print(f"F1-score: {np.mean(scores)} +- {np.std(scores)}; {np.mean(times)}")

F1-score: 0.9917494066241483 +- 0.00022626228787424871; 338.71823971271516
